### Load cross validation and testing data for normal vs COVID

In [6]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings("ignore")

In [7]:
# Load training data (one patient at a time)
num_folds = 5
X_train = [[] for x in range(num_folds)] # make list of arrays to hold X data for each cell
X_validation = [[] for x in range(num_folds)] # make list of arrays to hold X data for each cell
y_train = [[] for x in range(num_folds)] # make list of arrays to hold X data for each cell
y_validation = [[] for x in range(num_folds)] # make list of arrays to hold X data for each cell

X_test = pd.read_csv('normal_vs_disease_test_data.csv')
y_test = pd.read_csv('normal_vs_disease_test_label.csv')
y_test = np.array(y_test.drop(columns=['Unnamed: 0']))

#for i in range(num_folds):
for i in range(0, 1):
    X_filename =  str(i+1) + '_fold_training' + '.csv'
    y_filename = str(i+1) + '_fold_training_label' + '.csv'

    X_train_temp = pd.read_csv(X_filename)
    y_train_temp = pd.read_csv(y_filename)

    # Drop unnecessary columns
    y_train_temp = np.array(y_train_temp.drop(columns=['Unnamed: 0']))

    
    X_filename2 =  str(i+1) + '_fold_testing' + '.csv'
    y_filename2 = str(i+1) + '_fold_testing_label' + '.csv'

    X_validation_temp = pd.read_csv(X_filename2)
    y_validation_temp = pd.read_csv(y_filename2)

    # Drop unnecessary columns
    y_validation_temp = np.array(y_validation_temp.drop(columns=['Unnamed: 0']))

    
    X_train[i] = X_train_temp
    y_train[i] = y_train_temp
    X_validation[i] = X_validation_temp
    y_validation[i] = y_validation_temp

### Use the union of genes from top 100 DEGs of normal vs COVID and mild vs severe

In [8]:
top_gene2=[]

with open ("union_mild_severe_and_normal_rest_new.txt") as f2:
    lines2=f2.readlines()
    for line2 in lines2:
        line2=line2.strip()
        top_gene2.append(line2)

index2 = X_train[0].columns.get_indexer(top_gene2) 
print(top_gene2)
print(index2)
print(len(index2))

['CLDN3', 'TRAV6', 'SGK1', 'IL4I1', 'GJA1', 'KRT78', 'FABP4', 'S100A9', 'CXCL9', 'CYR61', 'ATF3', 'CD2', 'SMIM22', 'PLA2G7', 'ISG20', 'KLF6', 'RNASE7', 'LAMC2', 'S100A12', 'KRT7', 'MAFB', 'FTL', 'SPRR1B', 'IFNL2', 'PTPRCAP', 'SFN', 'SERPINB4', 'SAA2-SAA4', 'SFTA2', 'CD38', 'FBXW10', 'IDO1', 'FNIP2', 'TRBV11-3', 'C19orf33', 'GCH1', 'PEG10', 'IL21', 'CTSB', 'NUPR1', 'IGLV5-48', 'IFIT1', 'CTSL', 'ANKRD22', 'CTSE', 'CRYBA4', 'CCL2', 'CD7', 'SLAMF7', 'SPOCK2', 'HKDC1', 'HES4', 'FFAR2', 'C1QA', 'SPON1', 'IER3', 'SERPINB5', 'DUOX2', 'MARCKS', 'TNFSF13B', 'TRBV7-4', 'C1QB', 'FGL2', 'VAMP5', 'AQP1', 'AC114291.1', 'TGFBI', 'MAL2', 'TSHZ2', 'CEACAM7', 'ADGRF5', 'IGHV3-69-1', 'PRSS3', 'SFTPB', 'NR1H3', 'TNFSF10', 'PFKFB3', 'ANXA3', 'CXCL11', 'SPHK1', 'TYROBP', 'IGFL1', 'S100A14', 'CEACAM6', 'AC242022.1', 'IGFBP4', 'CD3E', 'IGHV5-51', 'PRSS22', 'TRDV3', 'CD48', 'NINJ1', 'CD1E', 'IFNL3', 'IL1RN', 'ICAM1', 'TRBV10-1', 'CTSD', 'TMPRSS11A', 'WIF1', 'GZMA', 'CCL8', 'SDS', 'C1QC', 'ADGRF4', 'TRAV38-2DV8'

In [17]:
SVM_f1_cv_new=[]
rf_f1_cv_new=[]
feature_importance_values = []

#for i3 in range(num_folds):
for i3 in range(0, 1):
    print("fold "+str(i3+1))
    X_train_new2 = X_train[i3].iloc[:, index2] # retain only the top n features in the data
    X_validation_new2 = X_validation[i3].iloc[:, index2] # retain only the top n features in the data
    
    clf_rf2 = RandomForestClassifier(n_estimators = 100)
    clf_rf2.fit(X_train_new2, y_train[i3])
    validation_pred2_rf = clf_rf2.predict(X_validation_new2)
    train_pred2_rf = clf_rf2.predict(X_train_new2)
    df = pd.DataFrame(train_pred2_rf)
    df.to_csv("train_pred2_rf_label.csv")
    df2 = pd.DataFrame(validation_pred2_rf)
    df2.to_csv("validation_pred2_rf_label.csv")
    print(len(train_pred2_rf))
    print(type(train_pred2_rf))
    print(len(validation_pred2_rf))
    print(type(validation_pred2_rf))


fold 1
13916
<class 'numpy.ndarray'>
3476
<class 'numpy.ndarray'>


In [18]:
X_test_new2 = X_test.iloc[:, index2] # retain only the top n features in the data
test_pred2_rf = clf_rf2.predict(X_test_new2)
df3 = pd.DataFrame(test_pred2_rf)
df3.to_csv("test_pred2_rf_label.csv")

### Load cross validation and testing data for mild vs severe using the union of genes from top 100 DEGs of normal vs COVID and mild vs severe

In [9]:
num_folds = 5

X_train2 = [[] for x in range(num_folds)] # make list of arrays to hold X data for each cell
X_validation2 = [[] for x in range(num_folds)] # make list of arrays to hold X data for each cell
y_train2 = [[] for x in range(num_folds)] # make list of arrays to hold X data for each cell
y_validation2 = [[] for x in range(num_folds)] # make list of arrays to hold X data for each cell

X_test2 = pd.read_csv('mild_vs_severe_test_data.csv')
y_test2 = pd.read_csv('mild_vs_severe_test_label.csv')
y_test2 = np.array(y_test2.drop(columns=['Unnamed: 0']))

#for i4 in range(num_folds):
for i4 in range(0, 1):
    X_filename3 =  str(i4+1) + '_fold_training_new' + '.csv'
    y_filename3 = str(i4+1) + '_fold_training_label_new' + '.csv'

    X_train_temp2 = pd.read_csv(X_filename3)
    y_train_temp2 = pd.read_csv(y_filename3)

    # Drop unnecessary columns
    y_train_temp2 = np.array(y_train_temp2.drop(columns=['Unnamed: 0']))

    
    X_filename4 =  str(i4+1) + '_fold_testing_new' + '.csv'
    y_filename4 = str(i4+1) + '_fold_testing_label_new' + '.csv'

    X_validation_temp2 = pd.read_csv(X_filename4)
    y_validation_temp2 = pd.read_csv(y_filename4)

    # Drop unnecessary columns
    y_validation_temp2 = np.array(y_validation_temp2.drop(columns=['Unnamed: 0']))
    
    X_train2[i4] = X_train_temp2
    y_train2[i4] = y_train_temp2
    X_validation2[i4] = X_validation_temp2
    y_validation2[i4] = y_validation_temp2


SVM_f1_cv_new2=[]
rf_f1_cv_new2=[]
feature_importance_values_new = []

#for i6 in range(num_folds):
for i6 in range(0, 1):
    print("fold "+str(i6+1))
    X_train_new4 = X_train2[i6].iloc[:, index2] # retain only the top n features in the data
    X_validation_new4 = X_validation2[i6].iloc[:, index2] # retain only the top n features in the data
    
    clf_rf4 = RandomForestClassifier(n_estimators = 100)
    clf_rf4.fit(X_train_new4, y_train2[i6])
    validation_pred4_rf = clf_rf4.predict(X_validation_new4)
    train_pred4_rf = clf_rf4.predict(X_train_new4)
    df4 = pd.DataFrame(train_pred4_rf)
    df4.to_csv("train_pred4_rf_label.csv")
    df5 = pd.DataFrame(validation_pred4_rf)
    df5.to_csv("validation_pred4_rf_label.csv")

fold 1


In [10]:
X_test_new4 = X_test2.iloc[:, index2] # retain only the top n features in the data
test_pred4_rf = clf_rf4.predict(X_test_new4)
df6 = pd.DataFrame(test_pred4_rf)
df6.to_csv("test_pred4_rf_label.csv")